### Imports

In [ ]:
import sympy as sym
import numpy as np
from numpy.random import randn # Gaussian random number

### Derivative of y

In [ ]:
x1, x2 = sym.symbols('x1 x2') 

n = 500
sigma = 10
sigma2 = 2*sigma**2
lam = 0.000001

x = randn(n)/2

x1_example = randn(n)/2
x2_example = randn(n)/2

noise = n * 1e-1
y1 = 1+ sym.sin(x1/10) + x1**2 + noise
y2 = 0.5 * x2**2 + sym.exp(x2) + noise

# derivate y1, y2 with respect to x1, x2 (sympy)
y1_diff = y1.diff(x1)
y2_diff = y2.diff(x2)

# Put back to numpy functions
y1_derivative = sym.lambdify(x1, y1_diff, 'numpy') 
y2_derivative = sym.lambdify(x2, y2_diff, 'numpy')

# Apply derivative functions to the x-arrays that we have in the beginning
y1_derivative = y1_derivative(x1_example)
y2_derivative = y2_derivative(x2_example)

y_derivative = np.array(np.ravel([y1_derivative, y2_derivative], 'F'))

### Derivate of f(x)

In [ ]:
# x,z = [x[:,i], x[:,j]]
# sym.exp(-((sym.sqrt(sympv.dot(x-z, x-z)))**2)/(2*sigma**2)) (with sympy)

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.array([x1, x2])

def k_gauss(x, z):
    k = np.exp(-((np.linalg.norm(x-z))**2)/(2*sigma**2))
    return k

# Derivative of the gaussian kernel
def k_gauss_derivative(x, z):
    kernel_gauss = k_gauss(x, z) # I formel er x - z tall, ikke vektorer
    k_derivative = kernel_gauss*(x-z)/sigma**2
    return k_derivative

# Alpha for gauss_kernel
def alpha_gauss_derivative(x, y, n):
    K = k_gauss_derivative(x, x, n)
    return np.linalg.inv(K + lam*np.eye(n)) @ y

def func_gauss_derivative(x, n, alpha):
    x_sort = np.sort(x)
    f = np.zeros(n)
    for i in range(n):
        sum = 0
        for j in range(n):
            sum = sum + alpha[j] * k_gauss_derivative(x[j], x_sort[i])
        f[i] = sum
    return f

alpha = alpha_gauss_derivative(x, y1_derivative, n)
func = func_gauss_derivative(x, n, alpha)
func